In [1]:
import pytest

import numpy as np

from unittest.mock import Mock
from torchinfo import summary

import torch

from tensorflow.python.platform import test

# from deepcell.model_zoo import PanopticNet
from panoptic import PanopticNet
from mesmer import Mesmer
from deepcell.applications import MultiplexSegmentation
from deepcell.applications.mesmer import format_output_mesmer
from deepcell.applications.mesmer import mesmer_postprocess
from deepcell.applications.mesmer import mesmer_preprocess

In [2]:
class TestMesmer(test.TestCase):

    def test_mesmer_app(self, load_path=None):
        with self.cached_session():
            whole_cell_classes = [1, 3]
            nuclear_classes = [1, 3]
            num_semantic_classes = whole_cell_classes + nuclear_classes
            num_semantic_heads = len(num_semantic_classes)

            device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
            if load_path is not None:
                model = torch.load(load_path, weights_only=False)
                model = model.to(device)
            else:
                model = PanopticNet(
                    'resnet50',
                    input_shape=(256, 256, 2),
                    norm_method=None,
                    num_semantic_heads=num_semantic_heads,
                    num_semantic_classes=num_semantic_classes,
                    location=True,
                    include_top=True,
                    use_imagenet=False,
                    device=device,
                )
                model = model.to(device)

            model.eval()

            with torch.no_grad():
                app = Mesmer(model)                
    
                # test output shape
                # shape = app.model.output_shape
                # self.assertIsInstance(shape, list)
                # self.assertEqual(len(shape), 4)
    
                # test predict with default
                x = np.random.rand(1, 500, 500, 2)
                # x = np.random.rand(1, 2, 500, 500)
                y = app.predict(x)
                self.assertEqual(x.shape[:-1], y.shape[:-1])
    
                # test predict with nuclear compartment only
                x = np.random.rand(1, 500, 500, 2)
                y = app.predict(x, compartment='nuclear')
                self.assertEqual(x.shape[:-1], y.shape[:-1])
                self.assertEqual(y.shape[-1], 1)
    
                # test predict with cell compartment only
                x = np.random.rand(1, 500, 500, 2)
                y = app.predict(x, compartment='whole-cell')
                self.assertEqual(x.shape[:-1], y.shape[:-1])
                self.assertEqual(y.shape[-1], 1)
    
                # test predict with both cell and nuclear compartments
                x = np.random.rand(1, 500, 500, 2)
                y = app.predict(x, compartment='both')
                self.assertEqual(x.shape[:-1], y.shape[:-1])
                self.assertEqual(y.shape[-1], 2)
    
                # test that kwargs are passed through successfully
                app._predict_segmentation = Mock()
    
                # get defaults
                _ = app.predict(x, compartment='whole-cell')
                args = app._predict_segmentation.call_args[1]
                default_cell_kwargs = args['postprocess_kwargs']['whole_cell_kwargs']
                default_nuc_kwargs = args['postprocess_kwargs']['nuclear_kwargs']
    
                # change one of the args for each compartment
                maxima_threshold_cell = default_cell_kwargs['maxima_threshold'] + 0.1
                radius_nuc = default_nuc_kwargs['radius'] + 2
    
                _ = app.predict(x, compartment='whole-cell',
                                postprocess_kwargs_whole_cell={'maxima_threshold':
                                                               maxima_threshold_cell},
                                postprocess_kwargs_nuclear={'radius': radius_nuc})
    
                args = app._predict_segmentation.call_args[1]
                cell_kwargs = args['postprocess_kwargs']['whole_cell_kwargs']
                assert cell_kwargs['maxima_threshold'] == maxima_threshold_cell
    
                nuc_kwargs = args['postprocess_kwargs']['nuclear_kwargs']
                assert nuc_kwargs['radius'] == radius_nuc
    
                # check that rest are unchanged
                cell_kwargs['maxima_threshold'] = default_cell_kwargs['maxima_threshold']
                assert cell_kwargs == default_cell_kwargs
    
                nuc_kwargs['radius'] = default_nuc_kwargs['radius']
                assert nuc_kwargs == default_nuc_kwargs
    
                # test legacy version
                # Change to cpu device in order to not break for now
                cpu_device = torch.device("cpu")
                cpu_model = model.to(cpu_device)
                cpu_model.device = cpu_device
                old_app = MultiplexSegmentation(cpu_model)
    
                # test predict with default
                x = np.random.rand(1, 500, 500, 2)
                y = old_app.predict(x)
                self.assertEqual(x.shape[:-1], y.shape[:-1])    
                print("SUCCESS")


In [3]:
a = TestMesmer()
a.test_mesmer_app()

channels_first
conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc
size of all 5
['P7', 'P6', 'P5', 'P4', 'P3']


/notebooks/torch-mesmer/deep_watershed.py:169: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_image = remove_small_objects(label_image,


SUCCESS


/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:193: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_image = remove_small_objects(label_image,
